#### IMPORTANT
Ensure you are utilizing 64-bit REFPROP with 64-bit python. If using the free version of REFPROP (MINI-REFPROP), please use 32-bit python and make changes to match the location where MINI-REFPROP is installed and make changes to the REFPROPFunctionLibrary function to read the REFPROP.DLL file.

Information on REFPROP and functions can be found here: https://buildmedia.readthedocs.org/media/pdf/refprop-docs/latest/refprop-docs.pdf

### IMPORT PACKAGES & FUNCTIONS

In [1]:
# Dictate the environment's loctaion of REFPROP
import os
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'

In [2]:
# Import the main class from the Python library
from ctREFPROP.ctREFPROP import REFPROPFunctionLibrary

# Imports from conda-installable packages
import pandas as pd

# Import numpy
import numpy as np

# Import matplotlib for plotting
import matplotlib.pyplot as plt

# Import Math for common values such as PI
import math


In [3]:
# Instantiate the library, and use the environment variable to explicitly state which path we want to use.
# As mentioned above, this will be changed to call the correct REFPROP functions to be used
# with MINI-REFPROP and 32-bit python.
# If using MINI-REFPROP and 32-bit python please make the following changes
# RP = REFPROPFunctionLibrary('C:/Program FIles (x86)/MINI-REFPROP\\REFPROP.DLL')
RP = REFPROPFunctionLibrary(os.environ['RPPREFIX'])

In [4]:
# This will call which root directory that will be used for the program. 
RP.SETPATHdll(os.environ['RPPREFIX'])

In [5]:
# Get the unit system we want to use (Mass base SI gives units in
# K, Pa, kg, m, N, J, W, and s)
MASS_BASE_SI = RP.GETENUMdll(0, "MASS BASE SI").iEnum

### HEAT EXCHANGER CALCS

#### Design Parameters of Heat Exchanger

In [6]:
# Outline the parameters of the Heat Exchanger (i.e. Tube ID & OD, Pipe ID & OD, Mass Flow rates)
Tube_OD = 0.50 # [inch]
Tube_ID = 0.402 # [inch]

Tube_OD = Tube_OD * 0.0254 # [Convert inches to meters]
Tube_ID = Tube_ID * 0.0254 # [Convert inches to meters]

Pipe_ID = 2.323 # [Inch]
Pipe_ID = Pipe_ID * 0.0254 # [Convert inches to meters]

# Calculate the Hydraulic Diameter of Heat Exchanger
Hyd_Dia = (((math.pi) * Pipe_ID**2 / 4) - ((math.pi) * Tube_OD**2 / 4)) * 4 / ((math.pi) * Tube_OD + (math.pi) * Pipe_ID) # [meters]

print("Hydraulic Diameter =" , Hyd_Dia, "meters")

# Mass Flow rate of sCO2
m_dot_sCO2 = 0.20 # (kg/s)

# Thermal Conductivity of the 316 S.S. tube [W/(m*K)]
Tube_Therm = 13.4

# Thermal Resistance of the Tube
R_cond = (math.log((Tube_OD/2)/(Tube_ID/2))) / Tube_Therm # [(m*K)/W]
R_cond

Hydraulic Diameter = 0.046304200000000004 meters


0.016280299239042582

#### sCO2 Properties

In [7]:
# Specify inlet conditions

P_in = 1245 # [psia]
T_in = 40.3 # [Celsius]

P_in = P_in * 6894.8 # convert psi to Pa
T_in = T_in + 273.15 # convert Celsius to Kelvin

In [8]:
# Obtain fluid properties from inlet conditions

sCO2_inlet = RP.REFPROPdll("CO2","PT","D;H;S;TCX;VIS;PRANDTL", MASS_BASE_SI,0,0,P_in,T_in,[1.0])

# Outputs will be placed into data frame for organization
sCO2_inlet = pd.DataFrame(sCO2_inlet.Output[0:6],
            index = ['Density [kg/m^3]', 'Enthalpy [J/kg]', 'Entropy [J/kg]',
                     'Thermal Cond. [W/(mK)]', 'Viscosity [Pa-s]', 'Prandtl'],
            columns = ['Inlet sCO2'])

# Display the data frame
sCO2_inlet

,Inlet sCO2
Density [kg/m^3],363.538420
Enthalpy [J/kg],376164.045515
Entropy [J/kg],1566.556525
Thermal Cond. [W/(mK)],0.059255
Viscosity [Pa-s],0.000026
Prandtl,4.178243


In [9]:
# Seperate the Data frame into workable 
sCO2_inlet.loc['Density [kg/m^3]','Inlet sCO2']

363.5384197606836

In [10]:
# Specify the desired outlet conditons 

P_out = 1200 # [psia] (This value is estimated and will be found later)
T_out = 35.5 # [Celsius]

P_out = P_out * 6894.8 # convert psi to Pa
T_out = T_out + 273.15 # convert Celsius to Kelvin

In [11]:
# Obtain fluid properties from the desired outlet conditions

sCO2_outlet = RP.REFPROPdll("CO2","PT","D;H;S;TCX;VIS;PRANDTL", MASS_BASE_SI,0,0,P_out,T_out,[1.0])

# Outputs will be placed into data frame for organization
sCO2_outlet = pd.DataFrame(sCO2_outlet.Output[0:6],
            index = ['Density [kg/m^3]', 'Enthalpy [J/kg]', 'Entropy [J/kg]',
                     'Thermal Cond. [W/(mK)]', 'Viscosity [Pa-s]', 'Prandtl'],
            columns = ['Outlet sCO2'])

# Display the data frame
sCO2_outlet

,Outlet sCO2
Density [kg/m^3],531.212789
Enthalpy [J/kg],326371.526533
Entropy [J/kg],1408.620082
Thermal Cond. [W/(mK)],0.082021
Viscosity [Pa-s],0.000038
Prandtl,8.968380


In [12]:
# Combine both data frames (will be used to call data for analysis)
sCO2 = pd.concat([sCO2_inlet, sCO2_outlet], axis =1)

# Display the data frame to ensure proper layout
sCO2

,Inlet sCO2,Outlet sCO2
Density [kg/m^3],363.538420,531.212789
Enthalpy [J/kg],376164.045515,326371.526533
Entropy [J/kg],1566.556525,1408.620082
Thermal Cond. [W/(mK)],0.059255,0.082021
Viscosity [Pa-s],0.000026,0.000038
Prandtl,4.178243,8.968380


In [13]:
# Find fluid properties at the mean temperature
P_mean = (P_in + P_out)/2 # [Pa] 
T_mean = (T_in + T_out)/2 # [Kelvin]


In [14]:
# Obtain fluid properties from the mean pressure and temperature

sCO2_mean = RP.REFPROPdll("CO2","PT","D;H;S;TCX;VIS;PRANDTL", MASS_BASE_SI,0,0,P_mean,T_mean,[1.0])

# Outputs will be placed into data frame for organization
sCO2_mean = pd.DataFrame(sCO2_mean.Output[0:6],
            index = ['Density [kg/m^3]', 'Enthalpy [J/kg]', 'Entropy [J/kg]',
                     'Thermal Cond. [W/(mK)]', 'Viscosity [Pa-s]', 'Prandtl'],
            columns = ['Mean sCO2'])

# Display the data frame
sCO2_mean

,Mean sCO2
Density [kg/m^3],413.280250
Enthalpy [J/kg],358563.904081
Entropy [J/kg],1511.460864
Thermal Cond. [W/(mK)],0.072196
Viscosity [Pa-s],0.000029
Prandtl,6.634124


In [15]:
# Combine into the previous data frame
sCO2 = pd.concat([sCO2, sCO2_mean], axis =1)

# Display the data frame to ensure proper layout
sCO2

,Inlet sCO2,Outlet sCO2,Mean sCO2
Density [kg/m^3],363.538420,531.212789,413.280250
Enthalpy [J/kg],376164.045515,326371.526533,358563.904081
Entropy [J/kg],1566.556525,1408.620082,1511.460864
Thermal Cond. [W/(mK)],0.059255,0.082021,0.072196
Viscosity [Pa-s],0.000026,0.000038,0.000029
Prandtl,4.178243,8.968380,6.634124


In [16]:
# Find the velocity at each of the conditions.
# This will then be used to find the respective Reynolds Number, Nusselt Number,
# and heat transfer coefficient (W/(m^2 * K))

Inlet_vel = (4 * m_dot_sCO2) / (sCO2.loc['Density [kg/m^3]', 'Inlet sCO2'] * (math.pi) * Tube_ID**2) # [m/s]
Outlet_vel = (4 * m_dot_sCO2) / (sCO2.loc['Density [kg/m^3]', 'Outlet sCO2'] * (math.pi) * Tube_ID**2) # [m/s]
Mean_vel = (4 * m_dot_sCO2) / (sCO2.loc['Density [kg/m^3]', 'Mean sCO2'] * (math.pi) * Tube_ID**2) # [m/s]

In [17]:
# Find the Reynolds Number
Inlet_Rey = sCO2.loc['Density [kg/m^3]', 'Inlet sCO2'] * Inlet_vel * Tube_ID / sCO2.loc['Viscosity [Pa-s]', 'Inlet sCO2']
Outlet_Rey = sCO2.loc['Density [kg/m^3]', 'Outlet sCO2'] * Outlet_vel * Tube_ID / sCO2.loc['Viscosity [Pa-s]', 'Outlet sCO2']
Mean_Rey = sCO2.loc['Density [kg/m^3]', 'Mean sCO2'] * Mean_vel * Tube_ID / sCO2.loc['Viscosity [Pa-s]', 'Mean sCO2']

In [18]:
# Find the Nusselt Number
Inlet_Nus = 0.0265 * Inlet_Rey**(4/5) * (sCO2.loc['Prandtl', 'Inlet sCO2'])**(0.3)
Outlet_Nus = 0.0265 * Outlet_Rey**(4/5) * (sCO2.loc['Prandtl', 'Outlet sCO2'])**(0.3)
Mean_Nus = 0.0265 * Mean_Rey**(4/5) * (sCO2.loc['Prandtl', 'Mean sCO2'])**(0.3)

In [19]:
# Using Nusselt Number, find the Heat transfer Coefficient (W/(m^2 * K))
Inlet_h_sCO2 = Inlet_Nus * (sCO2.loc['Thermal Cond. [W/(mK)]', 'Inlet sCO2']) / Tube_ID
Outlet_h_sCO2 = Outlet_Nus * (sCO2.loc['Thermal Cond. [W/(mK)]', 'Outlet sCO2']) / Tube_ID
Mean_h_sCO2 = Mean_Nus * (sCO2.loc['Thermal Cond. [W/(mK)]', 'Mean sCO2']) / Tube_ID

#### Water Properties

In [20]:
# Specify inlet conditions

P_in_water = 14.7 # [psia]
T_in_water = 25 # [Celsius]

P_in_water = P_in_water * 6894.8 # convert psi to Pa
T_in_water = T_in_water + 273.15 # convert Celsius to Kelvin

In [21]:
# Obtain fluid properties from inlet conditions

Water_inlet = RP.REFPROPdll("Water","PT","D;H;S;TCX;VIS;PRANDTL", MASS_BASE_SI,0,0,P_in_water,T_in_water,[1.0])

# Outputs will be placed into data frame for organization
Water_inlet = pd.DataFrame(Water_inlet.Output[0:6],
            index = ['Density [kg/m^3]', 'Enthalpy [J/kg]', 'Entropy [J/kg]',
                     'Thermal Cond. [W/(mK)]', 'Viscosity [Pa-s]', 'Prandtl'],
            columns = ['Inlet Water'])

# Display the data frame
Water_inlet

,Inlet Water
Density [kg/m^3],997.047650
Enthalpy [J/kg],104920.146257
Entropy [J/kg],367.199635
Thermal Cond. [W/(mK)],0.606516
Viscosity [Pa-s],0.000890
Prandtl,6.135805


In [22]:
# Using Energy Balance, find the outlet Enthalpy of the water at a specified 
# flow rate of water

Flow_water = 20 # gallons per minute of water
Flow_water = Flow_water * 0.00378541 # Convert gallons to meters cubed
m_dot_water = Flow_water / 60 * Water_inlet.loc['Density [kg/m^3]', 'Inlet Water']
m_dot_water

1.2580780478129785

In [23]:
# Conduct Energy Balance and find the outlet enthalpy

Water_Outlet_Enth = ((m_dot_sCO2 * (sCO2.loc['Enthalpy [J/kg]', 'Inlet sCO2'] - sCO2.loc['Enthalpy [J/kg]', 'Outlet sCO2']) / m_dot_water + Water_inlet.loc['Enthalpy [J/kg]', 'Inlet Water']))
Water_Outlet_Enth # [J/kg]

112835.79490337944

In [24]:
# Obtain fluid properties for outlet conditions

Water_outlet = RP.REFPROPdll("Water","PH","T;D;S;TCX;VIS;PRANDTL", MASS_BASE_SI,0,0,P_in_water,Water_Outlet_Enth,[1.0])

# Outputs will be placed into data frame for organization
Water_outlet = pd.DataFrame(Water_outlet.Output[0:6],
            index = ['Temperature [K]', 'Density [kg/m^3]', 'Entropy [J/kg]',
                     'Thermal Cond. [W/(mK)]', 'Viscosity [Pa-s]', 'Prandtl'],
            columns = ['Outlet Water'])

# Display the data frame
Water_outlet

,Outlet Water
Temperature [K],300.043264
Density [kg/m^3],996.545091
Entropy [J/kg],393.664914
Thermal Cond. [W/(mK)],0.609569
Viscosity [Pa-s],0.000853
Prandtl,5.849624


In [25]:
# Combine both data frames (will be used to call data for analysis)
Water = pd.concat([Water_inlet, Water_outlet], axis =1)

# Add data into the data frame
Water.loc['Enthalpy [J/kg]', 'Outlet Water'] = Water_Outlet_Enth
Water.loc['Temperature [K]', 'Inlet Water'] = T_in_water
# Display the data frame to ensure proper layout
Water

,Inlet Water,Outlet Water
Density [kg/m^3],997.047650,996.545091
Enthalpy [J/kg],104920.146257,112835.794903
Entropy [J/kg],367.199635,393.664914
Thermal Cond. [W/(mK)],0.606516,0.609569
Viscosity [Pa-s],0.000890,0.000853
Prandtl,6.135805,5.849624
Temperature [K],298.150000,300.043264


In [26]:
# Find the mean Velocity of Water flowing
Water_vel = 4 * m_dot_water / (((Water.loc['Density [kg/m^3]', 'Inlet Water'] + Water.loc['Density [kg/m^3]', 'Outlet Water'])/2) \
             * math.pi * Hyd_Dia**2)

Water_vel # [m/s]

0.7494980107528856

In [27]:
# Find the Reynolds Number
Water_Rey = ((Water.loc['Density [kg/m^3]', 'Inlet Water'] + Water.loc['Density [kg/m^3]', 'Outlet Water'])/2) * \
            Water_vel * Hyd_Dia / ((Water.loc['Viscosity [Pa-s]', 'Inlet Water'] + Water.loc['Viscosity [Pa-s]', 'Outlet Water'])/2)

Water_Rey

39695.70958456734

In [28]:
# Find the Nusselt Number
Water_Nus = 0.0243 * Water_Rey**(4/5) * \
            ((Water.loc['Prandtl', 'Inlet Water'] + Water.loc['Prandtl', 'Outlet Water'])/2)**(0.4)

Water_Nus

237.49286053874806

In [29]:
# Using Nusselt Number, find the Heat transfer Coefficient (W/(m^2 * K))
h_Water = Water_Nus * ((Water.loc['Thermal Cond. [W/(mK)]', 'Inlet Water'] + Water.loc['Thermal Cond. [W/(mK)]', 'Outlet Water'])/2)\
            / Hyd_Dia

h_Water

3118.63120312362

#### Analysis of Heat Exchanger

In [30]:
# Find the log mean temperature difference
Delta_T1 = T_in - Water.loc['Temperature [K]', 'Outlet Water']
Delta_T2 = T_out - Water.loc['Temperature [K]', 'Inlet Water']

Log_Mean_T = (Delta_T2 - Delta_T1) / math.log(Delta_T2/Delta_T1)
Log_Mean_T

11.894230868248954

In [31]:
# Approximate resistance of the cylindrical tube using the slab formula 
# L/(kA)

Rt_cond_approx = (Tube_OD - Tube_ID) / (2 * Tube_Therm) # [(m^2 * K) / W]
Rt_conv_sCO2 = 1 / Inlet_h_sCO2 # [(m^2 * K) / W]
Rt_conv_Water = 1 / h_Water

U_inlet = 1 / (Rt_cond_approx + Rt_conv_sCO2 + Rt_conv_Water)
U_inlet

2069.9225174819185

In [32]:
# Find the Length of tube neccessary for the Heat Exchanger
Length = (m_dot_sCO2 * (sCO2.loc['Enthalpy [J/kg]', 'Inlet sCO2'] - sCO2.loc['Enthalpy [J/kg]', 'Outlet sCO2']))\
          / (U_inlet * Log_Mean_T * math.pi * Tube_OD)

Length = Length * 3.28084 # Converts meters to feet
Length # This length uses inlet conditions of sCO2 to approximate length in feet

33.26098434946828

In [33]:
# Approximate resistance of the cylindrical tube using the slab formula 
# L/(kA)

Rt_cond_approx = (Tube_OD - Tube_ID) / (2 * Tube_Therm) # [(m^2 * K) / W]
Rt_conv_sCO2 = 1 / Outlet_h_sCO2 # [(m^2 * K) / W]
Rt_conv_Water = 1 / h_Water

U_outlet = 1 / (Rt_cond_approx + Rt_conv_sCO2 + Rt_conv_Water)
U_outlet

2140.1795979146245

In [34]:
Length = (m_dot_sCO2 * (sCO2.loc['Enthalpy [J/kg]', 'Inlet sCO2'] - sCO2.loc['Enthalpy [J/kg]', 'Outlet sCO2']))\
          / (U_outlet * Log_Mean_T * math.pi * Tube_OD)

Length = Length * 3.28084 # Converts meters to feet
Length # This length uses outlet conditions of sCO2 to approximate length in feet

32.169104184369736

In [35]:
# Approximate resistance of the cylindrical tube using the slab formula 
# L/(kA)

Rt_cond_approx = (Tube_OD - Tube_ID) / (2 * Tube_Therm) # [(m^2 * K) / W]
Rt_conv_sCO2 = 1 / Mean_h_sCO2 # [(m^2 * K) / W]
Rt_conv_Water = 1 / h_Water

U_mean = 1 / (Rt_cond_approx + Rt_conv_sCO2 + Rt_conv_Water)
U_mean

2138.865162063363

In [36]:
Length = (m_dot_sCO2 * (sCO2.loc['Enthalpy [J/kg]', 'Inlet sCO2'] - sCO2.loc['Enthalpy [J/kg]', 'Outlet sCO2']))\
          / (U_mean * Log_Mean_T * math.pi * Tube_OD)

Length = Length * 3.28084 # Converts meters to feet
Length # This length uses mean conditions of sCO2 to approximate length in feet

32.18887365118461